# `libcusmm`: Explore the Training Data

This notebook allows you to explore the training data collected from autotuning before proceeding to training.

### Import libraries

In [ ]:
import re, sys, os, json, random
import numpy as np
import pandas as pd
import dask.dataframe as dd
from nb_helper import *

### Path to autotuning data

Provide the path to the autotuning data:
- You can use the bash cell below to navigate your filetree:

In [ ]:
%%bash
ls -ad AUTOTUNING_DATA_PATH/tune_*x*x*/

- Then, copy what you've replaced `AUTOTUNING_DATA_PATH` with in the Python variable `autotuning_data_path` below:

In [ ]:
autotuning_data_path = '' # may not recognize '~', please provide an absolute path:
check_autotuning_data_path(autotuning_data_path)

### Set options

Set the following options appropriately:

In [ ]:
to_read = 100       # How many / which data folders to read. Options: 
                    # - 'all': reads from all available data folders. 
                    #   Beware, this might result in memory errors if large amounts of data are made available
                    # - a number: reads this number of data folders (e.g. 100)
                    # - a regex: reads the data folders with matching regex (e.g. tune_4x*x*)

In [ ]:
# Get the list of folders to read
folders_to_read = get_folders_to_read(to_read, autotuning_data_path)

In [ ]:
algorithm = get_algorithm_to_explore('all')   # algorithms to explore. Options: all, tiny, small, medium

In [ ]:
thresh = 300000     # do not perform very long operations on row counts above this threshold

### Read training data

In [ ]:
files_to_read = get_files_to_read(folders_to_read, algorithm)

In [ ]:
%%time
num_f = len(files_to_read)
data = dd.read_csv(files_to_read)

### Data head

In [ ]:
for i in range(0, len(data.columns.values), page_width):
    display(data.iloc[:,i:i+page_width].head())

### Data description

In [ ]:
print('Data size        :', sys.getsizeof(data)/10**6, 'MB')
print('Number of columns:', len(data.columns.values))
print('Number of rows   : {:,}'.format(len(data.index)))

In [ ]:
#for i in range(0, len(data.columns.values), page_width):
#    display(data.iloc[:,i:i+page_width].describe())

### Columns

In [ ]:
#print('Number of columns:', len(data.columns), '\nNumber of rows:', len(data.index), '\n')
for col in data.columns: 
    print('{:<40} {}'.format(col, data[col].dtype))

In [ ]:
# Feature categories
mnk = ['m', 'n', 'k']
kernel_pars = ['algorithm', 'threads_per_blk', 'grouping', 'minblocks',
               'tile_m', 'tile_n', 'w', 'v', 'nbytes_smem', 'nbytes_cmem', 'regs_per_thread']
kernel_pars = list(set(kernel_pars) & set(data.columns.values))
perf =  ['perf (Gflop/s)', 'perf_squared', 'perf_scaled', 'perf_scaled_by_algo']
common = ['Gflops', 'mxnxk', 'size_a', 'size_b', 'size_c', 'nblks', 
          'warps_per_blk', 'nwarps', 'sm_desired', 'nthreads', 'ru_param_stack_unroll_factor']

### Sample row

In [ ]:
other_columns = list(set(data.columns.values) - set(kernel_pars + mnk + perf + common))
irow = 0#random.sample(range(len(data.index.values)), 1)[0]
row = data.iloc[irow,:]
print(row[mnk], '\n')
print(row[kernel_pars], '\n')
print(row[perf], '\n')
print(row[common], '\n')
print(row[other_columns], '\n')

### Features

Features in the left-most column correspond to "raw" parameters
* **green** kernel parameters 
* **grey** CUDA card properties (taken from CUDA documentation) 
* **pink** autotuning parameters (taken from DBCSR codebase) 

Other features correspond to derived parameters, computed from the "raw" parameters
* **yellow** matrix sizes
* **light grey** launch parameters
* **blue** and **purple** estimations of resource usages

![parameters dependency graph](libcusmm_parameters_and_memory.png)

In [ ]:
data_profiling = data
if len(data.index.values) > thresh:  # if it is a very large dataframe, perform op on subsampled rows 
    sampled_rows = random.sample(data_profiling.index.values.tolist(), thresh)
    data_profiling = data.iloc[sampled_rows,:]

import pandas_profiling 
pandas_profiling.ProfileReport(data_profiling)

### Data visualization

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.semilogx(data['mxnxk'], data['perf (Gflop/s)'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance [Gflops]')

In [ ]:
plt.semilogx(data['mxnxk'], data['perf_squared'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance squared [Gflops]^2')

### Data visualization (scaled performance)

In [ ]:
plt.plot(data['mxnxk'], data['perf_scaled'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance scaled (overall)')

In [ ]:
algos = np.unique(data['algorithm'])
for algo in algos:
    algo_data = data[data['algorithm'] == algo]
    plt.figure()
    plt.semilogx(algo_data['mxnxk'], algo_data['perf_scaled_by_algo'], '.', markersize=1)
    plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
    plt.ylabel('Performance scaled (by algorithm)')
    plt.title(algo)
    plt.show()

### Performance profile

In [ ]:
# Choose (m, n, k) triplet
m_plot, n_plot, k_plot = (4, 4, 4)

In [ ]:
data_mnk = data[data['m'] == m_plot][ 
                data['n'] == n_plot][ 
                data['k'] == k_plot]
data_mnk.sort_values(by='perf (Gflop/s)', ascending=True, inplace=True)
plt.plot(data_mnk['perf (Gflop/s)'].values)
plt.xlabel('parameter set')
plt.ylabel('perf (Gflop/s)')
plt.title('Performance profile for kernel ' + str(m_plot) + 'x'+ str(n_plot) + 'x'+ str(k_plot))

In [ ]:
# Histograms with Bokeh
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

# Create histogram
num_bins = 100 
hist, edges = np.histogram(data_mnk['perf (Gflop/s)'], bins=num_bins)
df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
source = ColumnDataSource(df_hist)

# Create tool 
hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

# Create the figure
p = figure(plot_width=800, plot_height=800, title="Performance histogram",
           toolbar_location=None, tools="")
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Performance (GFlop/s)"
p.xaxis.major_label_orientation = 1.2
p.yaxis.axis_label = "# occurrences"
p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
p.add_tools(hover)
show(p)


In [ ]:
# Histograms with Bokeh
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

# Create histogram
num_bins = 100 
hist, edges = np.histogram(data_mnk['perf_scaled'], bins=num_bins)
df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
source = ColumnDataSource(df_hist)

# Create tool 
hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

# Create the figure
p = figure(plot_width=800, plot_height=800, title="Performance histogram",
           toolbar_location=None, tools="")
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Performance scaled"
p.xaxis.major_label_orientation = 1.2
p.yaxis.axis_label = "# occurrences"
p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
p.add_tools(hover)
show(p)

In [ ]:
# Top slices of perf. distribution
pars_autotuning_top = {
    5: list(), 
    2: list(), 
    1: list(), 
    0.5: list()
}
max_perf = float(data_mnk['perf (Gflop/s)'].max())
max_perf_idx = data_mnk['perf (Gflop/s)'].idxmax()
max_perf_row = data_mnk.loc[max_perf_idx]
max_perf_cond = max_perf_row[mnk + kernel_pars + ['perf (Gflop/s)']]

print('Maximally performing parameter set:')
display(max_perf_cond)
for perc in pars_autotuning_top.keys():
    lim = max_perf - max_perf*perc/100
    blob = data_mnk.loc[data_mnk['perf (Gflop/s)'] >= lim]
    print('\ntop', perc, '%')
    display(blob[kernel_pars + ['perf (Gflop/s)']].describe())
    pars_autotuning_top[perc].append(blob)

### Pair plot 

In [ ]:
data_pairplot = data
if len(data.index.values) > thresh:  # if it is a very large dataframe, perform op on subsampled rows 
    sampled_rows = random.sample(data_pairplot.index.values.tolist(), thresh)
    data_pairplot = data.iloc[sampled_rows,:]

sns.pairplot(data_pairplot[mnk + kernel_pars + perf].dropna())